In [ ]:
import sys
!pip install pandas
!pip install numpy
!pip install datapane
!pip install plotly
!pip install pymysql

In [ ]:
# import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp
import pymysql
import warnings
import pymysql
import sys
import os

warnings.filterwarnings('ignore')



# **Connection to Database**

In [ ]:
host = 'database-1.us-east-2.rds.amazonaws.com'
user = 'admin'
password = '12345'
database = 'EDM_Project'

connection = pymysql.connect(host=host, user=user, password=password, database=database)
with connection:
    cur = connection.cursor()
    cur.execute("SELECT VERSION()")
    version = cur.fetchone()
    print("Database version: {} ".format(version[0]))

In [ ]:

def run_query(q: str) -> pd.DataFrame:
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        return pd.read_sql(q, conn)


def run_command(c: str):
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        conn.isolation_level = None
        conn.execute(c)

## **Testing the connection by printing the name and number of records for all the tables available in the database**

In [ ]:
def show_tables() -> pd.DataFrame:
    q = """
          SELECT table_name
          FROM information_schema.tables
          WHERE table_schema = 'EDM_Project' AND table_type = 'BASE TABLE';
        """
    return run_query(q)


def get_table_row_count(tablename) -> int:
    q = (
        """
        SELECT
            COUNT(1)
        FROM %s;
        """
        % tablename
    )
    return run_query(q)["COUNT(1)"][0]


tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables['TABLE_NAME']]

tables



,TABLE_NAME,row_count
0,Customer,50
1,Employee,10
2,Invoice,51
3,Product,10
4,Product_Line,90
5,Stock_on_hand,10
6,Store,10
7,Supplier,10


As you can see, we write the SQL query as a string and then execute it within a Python function. It’s a little awkward to mix two different languages together like this, but works surprisingly well once you get the hang of it! Running the code above gives all the different table names in the database and the number of rows in each table.

# Building a dashboard
To know the sales of each stores based on invoices generated so that they can make strategies accordingly. To answer this question, we’ll need to write a query like this:


In [ ]:
store_performance = """
SELECT
    S.Store_Name as 'Store Name', S.Store_Address, sum(I.Invoice_Total) as 'Total Sales in $'
    From Invoice I
    left join Store S on I.Store_ID = S.Store_ID
    group by S.Store_ID
    order by sum(I.Invoice_TOtal) desc

"""


store_sales = run_query(store_performance)

dp.DataTable(store_sales)

We start by selecting the columns we want to see in the output. Since these come from two different tables (inovice and store), we need to join the tables on the foreign key relationships — check the schema to see what is possible.

Next we group by the store id and order them by the most sales done.
Running the query gives us a pandas dataframe, which we then wrap in a DataTable block.

Flush with our recent success, the boss asks us to visualize this in bar graph, we use a Python visualization library like Plotly to build the bar graph:

In [ ]:
store_sales_bar = px.bar(
    store_sales, y="Store Name", x="Total Sales in $")

store_sales_bar.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

store_sales_bar.show()

## **Number of Customers in Each State**

> Indented block


In [ ]:
state_query = '''
    Select Customer_State as State, count(Customer_State) as 'Number Of Customers'
from Customer
Group by Customer_State
Order by count(Customer_State) desc

'''
state = run_query(state_query)
state

,State,Number Of Customers
0,FL,5
1,OH,5
2,MI,5
3,WA,4
4,CA,4
5,CO,4
6,IL,3
7,GA,3
8,MO,3
9,IN,3


In [ ]:
bar_state_customer_breakdown = px.bar(state,
    x = 'State',
    y = 'Number Of Customers',
    color = 'State',
    labels ={
        "State" : "State",
        "Number Of Customers": "Number of Custormers per State"
    },
    title = "Customers Per State")

bar_state_customer_breakdown.update_layout(title_text="Number of Customers in Each State")

bar_state_customer_breakdown.show()

#No. of Products by Category

In [ ]:
Products_by_Category = '''

select Product_Category, count(Product_ID) as 'Total Number of Products'
from Product
group by Product_Category
order by count(Product_ID) desc

'''
products = run_query(Products_by_Category)
products



,Product_Category,Total Number of Products
0,Power Tools,2
1,Computers,1
2,Audio,1
3,Computer Components,1
4,Smart Home,1
5,Lighting,1
6,Security,1
7,DIY Electronics,1
8,3D Printers,1


In [ ]:
products_pie = go.Figure(data=[go.Pie(
    labels = products["Product_Category"],
    values=products["Total Number of Products"],
    hole=.3)
                                 ])

products_pie.update_layout(title_text="Products by Category")

products_pie.show()

In [ ]:
count_of_products_sold_query = '''
select  P.Product_Category, P.Product_Name as 'Product Name' , count(pl.Line_Qty) as 'No. of Products'
from Invoice I
left join Product_Line pl on I.Invoice_ID = pl.Invoice_ID
left join Product P on pl.Product_ID = P.Product_ID
group by pl.Product_ID

'''

count_of_products_sold = run_query(count_of_products_sold_query)
count_of_products_sold

,Product_Category,Product Name,No. of Products
0,Computers,Tablet,9
1,Audio,Bluetooth Speakers,8
2,Power Tools,Circular Saw,10
3,Power Tools,Industrial Drill,9
4,Computer Components,Graphics Card,9
5,Smart Home,Smart Thermostat,9
6,Lighting,LED Desk Lamp,9
7,Security,Wireless Security Camera,9
8,DIY Electronics,Arduino Starter Kit,9
9,3D Printers,3D Printer,9


In [ ]:
count_of_products_sold_pie = go.Figure(data=[go.Pie(
    labels = count_of_products_sold["Product_Category"],
    values=count_of_products_sold["No. of Products"],
    hole=.3)
                                 ])

count_of_products_sold_pie.update_layout(title_text="Products sold Category wise")

count_of_products_sold_pie.show()